# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [2]:
import math

1/math.exp(0.9126)

0.4014790204046642

In [3]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [4]:
import urllib2
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [5]:
## YOUR CODE HERE
#resultIndeed = urllib2.urlopen(URL).read()
url = "http://www.indeed.com/jobs?q=data+scientist+%2490%2C000+-+%24200%2C000&l={}&start={}"


Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

In [6]:
## YOUR CODE HERE
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''
    
def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

def get_location_from_result(result):
    return  extract_text(result.find('span', {'class' : 'location'}))

def get_summary_from_result(result):
    return  extract_text(result.find('span', {'class' : 'summary'}))

def get_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()
        
    return None


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [7]:
rows = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 'Atlanta']):
    for start in range(1, 1000, 10):
        r = requests.get(url.format(city, start))
        soup = BeautifulSoup(r.content,"lxml")
        results = soup.findAll('div', { "class" : "result" })
        for result in results:
            if result:
                row = {}
                row['title'] = get_title_from_result(result)
                row['company'] = get_company_from_result(result)
                row['summary'] = get_summary_from_result(result)
                row['salary'] = get_salary_from_result(result)
                row['city'] = city
                rows.append(row)



#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [8]:
## Code completed above
## YOUR CODE HERE
data = pd.DataFrame.from_records(rows)

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [9]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [10]:
## YOUR CODE HERE

### Save your results as a CSV

In [11]:
## YOUR CODE HERE
data.to_csv("../assets/indeedproject.csv", index=False,encoding='utf-8')

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [12]:
## YOUR CODE HERE
import pandas as pd
salary_data = pd.read_csv("../assets/indeedproject.csv")
salary_data = salary_data[salary_data.salary.notnull()]
salary_data = salary_data[~(salary_data.salary.astype('str').str.contains('hour'))]

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [13]:
## YOUR CODE HERE
import numpy as np 
def cleanSalaryColumn(salary):
    badChars = ['a year','a month', " ", "$", "a",",","dy"]
    string = salary
    for x in badChars:
        string = string.replace(x,"")
    return string

def findBound(salary,boundIndex):
    if (salary == np.nan) | (salary == None):
        return salary
    else:
        if  "-" in salary:
            return salary.split("-")[boundIndex]
        return salary

salary_data['salary_as_num'] = salary_data.salary.apply(cleanSalaryColumn)
salary_data['salary_ub'] = salary_data.salary_as_num.apply( lambda x: findBound(x,1) ).astype(float)
salary_data['salary_lb'] = salary_data.salary_as_num.apply( lambda x: findBound(x,0) ).astype(float)


In [14]:
def getClass(x,threshold):
    ub = x[6]
    lb = x[7]
    sal = x[2]
    if 'month' in sal.lower():
        ub = float(ub * 12)
        lb = float(lb * 12)
        
    avg = float(( ub + lb ) / 2)
    if avg >= threshold:
        return 1
    else:
        return 0
    
def binOccupation(title):
    title = title.lower()
    
#    finalBin = 0
    if 'scientist' in title:
        finalBin =  1
    elif 'engineer' in title:
        finalBin =  2
    elif 'product' in title:
        finalBin =  3
    elif 'stat' in title:
        finalBin =  4
    elif 'analyst' in title:
        finalBin =  5
    elif ('software engineer' in title) | ('developer' in title):
        finalBin =  6
    else:
        finalBin = 0
   
    return finalBin

def binSeniority(title):
    title = title.lower()
  
#    finalBin = 0
    if ('sr' in title) | ('senior' in title):
        finalBin = 1
    elif ('jr' in title) | ('junior' in title):
        finalBin = 2
    elif ('mgr' in title) | ('manager' in title):
        finalBin = 2
    elif ('associate' in title) | ('ass' in title) | ('mid' in title):
        finalBin = 3
    elif 'vp' in title:
        finalBin = 4
    elif 'lead' in title:
        finalBin = 5
    elif 'director' in title:
        finalBin = 6
    else:
        finalBin = 0
    return finalBin
        
#def monthSalYear(x):
    
salary_data['occupationGrp'] = salary_data.title.apply(binOccupation).astype(int)
salary_data['seniorityGrp'] = salary_data.title.apply(binSeniority).astype(int)
salary_data['class'] = salary_data.apply( lambda x: getClass(x,130000), axis = 1 ).astype(int)
salary_data

,city,company,salary,summary,title,salary_as_num,salary_ub,salary_lb,occupationGrp,seniorityGrp,class
3,Chicago,Workbridge Associates,"$105,000 a year","Data, Analytics, Scientist, Science, Python, R...",Mid-Level Data Scientist,105000,105000.0,105000.0,1,3,0
12,Chicago,Workbridge Associates,"$80,000 - $110,000 a year",Big data with Hadoop is a plus. A well-respect...,Data Scientist,80000-110000,110000.0,80000.0,1,0,0
18,Chicago,All-In Analytics,"$115,000 - $150,000 a year",This role will be responsible for leveraging a...,Machine Learning Data Scientist,115000-150000,150000.0,115000.0,1,0,1
36,Chicago,Seeking Alpha Search International,"$110,000 - $200,000 a year",Working with data validation members and infor...,QUANTITATIVE MODEL VALIDATION ANALYST,110000-200000,200000.0,110000.0,5,0,1
49,Chicago,Workbridge Associates,"$150,000 a year",If you’re a Senior Data Scientist that thrives...,Senior Data Scientist,150000,150000.0,150000.0,1,1,1
65,Chicago,Hirewell,"$125,000 a year",Data manipulation with SQL systems. Manipulate...,Data Scientist,125000,125000.0,125000.0,1,0,0
69,Chicago,CREhire.com,"$138,300 a year",Categorical Data Analysis. Statistical Analysi...,Principal Statistical Analyst / Data Scientist,138300,138300.0,138300.0,1,0,1
72,Chicago,Workbridge Associates,"$145,000 a year",Hiring for a Senior Data Scientist with a stro...,Natural Language Processing Scientist,145000,145000.0,145000.0,1,0,1
78,Chicago,North shore Staffing Group,"$95,000 a year",Business Data Analyst – Mobile Devices to join...,Business Data Analyst,95000,95000.0,95000.0,5,0,0
97,Chicago,Workbridge Associates,"$140,000 a year","Senior, Lead, Data, Scientist, Scientists, Ana...",Lead Data Scientist,140000,140000.0,140000.0,1,5,1


In [15]:
colsToKeep = salary_data.columns[8:]
print colsToKeep
cleanDf = salary_data[colsToKeep]
dummies1 = pd.get_dummies(cleanDf.occupationGrp, prefix = "occupation")
dummies2 = pd.get_dummies(cleanDf.seniorityGrp, prefix = "seniority")
join1 = cleanDf[ [cleanDf.columns[2]] ].join( dummies1 )
join2 = join1.join( dummies2  )
#print cleanDf.columns[2] 
join2

Index([u'occupationGrp', u'seniorityGrp', u'class'], dtype='object')


,class,occupation_0,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,occupation_6,seniority_0,seniority_1,seniority_2,seniority_3,seniority_4,seniority_5,seniority_6
3,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
18,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
36,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
49,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
65,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
69,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
72,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
97,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


#### Thought experiment: What is the baseline accuracy for this model?

In [16]:
## YOUR CODE HERE
from sklearn.cross_validation import train_test_split
import statsmodels.formula.api as sm

featureCols = join2.columns[1:]
specialCol = join2.columns[:1]
X = join2[featureCols]
X = X[ X.columns[1:6] + X.columns[7:] ]
#y = join2['class']
y = join2[specialCol]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)
logit = sm.Logit(y_train, X_train)
result = logit.fit_regularized()
result.summary()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.449620974524
            Iterations: 104
            Function evaluations: 104
            Gradient evaluations: 104


/Users/SatyaSagi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  class   No. Observations:                  298
Model:                          Logit   Df Residuals:                      286
Method:                           MLE   Df Model:                           11
Date:                Thu, 07 Jul 2016   Pseudo R-squ.:                  0.2846
Time:                        10:39:51   Log-Likelihood:                -133.99
converged:                       True   LL-Null:                       -187.29
                                        LLR p-value:                 8.736e-18
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
occupation_1     1.0148      0.565      1.797      0.072        -0.092     2.122
occupation_2     2.4151      0.748      3.230      0.001         0.949     3.881
occupation_3   -16.3311   5091.942     -0.003      0.997     -9996.355  9963.692
occupation_4    -0.4477      1.236     -0.362      0.717        -2.870     1.974
occupation_5     1.5896      0.678      2.344      0.019         0.261     2.919
seniority_0     -0.7397      0.491     -1.506      0.132        -1.703     0.223
seniority_1     -2.2166      0.596     -3.720      0.000        -3.385    -1.049
seniority_2      0.8708      1.173      0.743      0.458        -1.428     3.170
seniority_3     -4.5912      0.910     -5.043      0.000        -6.376    -2.807
seniority_4     27.7064   3.51e+05   7.89e-05      1.000     -6.88e+05  6.88e+05
seniority_5     -2.5603      0.914     -2.801      0.005        -4.352    -0.769
seniority_6     -1.2528      0.567     -2.210      0.027        -2.364    -0.142
================================================================================
"""

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [17]:
## YOUR CODE HERE
logitO = sm.Logit(y, X)
resultO = logitO.fit_regularized()
resultO.summary()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.439929076074
            Iterations: 114
            Function evaluations: 114
            Gradient evaluations: 114


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  class   No. Observations:                  427
Model:                          Logit   Df Residuals:                      415
Method:                           MLE   Df Model:                           11
Date:                Thu, 07 Jul 2016   Pseudo R-squ.:                  0.2929
Time:                        10:39:51   Log-Likelihood:                -187.85
converged:                       True   LL-Null:                       -265.65
                                        LLR p-value:                 1.071e-27
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
occupation_1     1.4071      0.466      3.018      0.003         0.493     2.321
occupation_2     2.7344      0.655      4.174      0.000         1.450     4.018
occupation_3    -0.1421      1.753     -0.081      0.935        -3.577     3.293
occupation_4     0.8803      0.825      1.067      0.286        -0.736     2.497
occupation_5     1.7414      0.545      3.197      0.001         0.674     2.809
seniority_0     -0.9432      0.390     -2.420      0.016        -1.707    -0.179
seniority_1     -2.6600      0.498     -5.345      0.000        -3.635    -1.685
seniority_2      1.2273      1.212      1.013      0.311        -1.148     3.603
seniority_3     -5.3106      0.851     -6.240      0.000        -6.979    -3.643
seniority_4     18.0506   4508.711      0.004      0.997     -8818.861  8854.963
seniority_5     -2.7484      0.680     -4.042      0.000        -4.081    -1.416
seniority_6     -1.3399      0.436     -3.073      0.002        -2.195    -0.485
================================================================================
"""

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
    - or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [18]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [19]:
## YOUR CODE HERE
from sklearn.linear_model import LogisticRegression 
modelSk = LogisticRegression()
resultSk = modelSk.fit(X, y.values.ravel())
predictionSk = resultSk.predict(X)
print predictionSk

[0 1 1 1 0 1 1 1 1 0 0 0 0 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 1 1 0 1 0 1
 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 0 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 1
 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 0
 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0
 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0
 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0
 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [20]:
#print y
ravely = pd.Series(y.values.ravel())
ravely.name ="actual"
predictionSk = pd.Series(predictionSk)
predictionSk.name ="predict"
pd.crosstab(ravely, predictionSk)
#print len(y), len(predictionSk)


predict,0,1
actual,,
0,262,31
1,52,82


#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [21]:
## YOUR CODE HERE
import sklearn.metrics as met

print met.accuracy_score(y,predictionSk)
print met.recall_score(y, predictionSk)
print met.precision_score(y, predictionSk)

0.805620608899
0.611940298507
0.725663716814


#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [22]:
## YOUR CODE HERE
modelSk = LogisticRegression()
modelSk.penalty = "l1"
resultSk = modelSk.fit(X, y.values.ravel())
predictionSk = resultSk.predict(X)


In [23]:
#print y
ravely = pd.Series(y.values.ravel())
ravely.name ="actual"
predictionSk = pd.Series(predictionSk)
predictionSk.name ="predict"
pd.crosstab(ravely, predictionSk)
#print len(y), len(predictionSk)


predict,0,1
actual,,
0,264,29
1,49,85


In [24]:
## YOUR CODE HERE
modelSk = LogisticRegression()
modelSk.penalty = "l2"
resultSk = modelSk.fit(X, y.values.ravel())
predictionSk = resultSk.predict(X)


In [25]:
#print y
ravely = pd.Series(y.values.ravel())
ravely.name ="actual"
predictionSk = pd.Series(predictionSk)
predictionSk.name ="predict"
pd.crosstab(ravely, predictionSk)
#print len(y), len(predictionSk)

predict,0,1
actual,,
0,262,31
1,52,82


#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [26]:
## YOUR CODE HERE

In [27]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [28]:
## YOUR CODE HERE